# Setting

Mount your Google Drive

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Set up mount symlink

In [2]:
DRIVE_PATH = '/content/gdrive/My\ Drive/PointNet'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/PointNet'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

## Import and others

In [3]:
from pathlib import Path

import numpy as np
import torch
import torch.nn
import torch.nn.functional as F
import  matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


import time

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


## Download data
Original dataset takes too much time in reading files and sampling - so we did it beforehand.

It reduced training time from 40 mins per epoch to 1 mins per epoch!

In [6]:
%cd $SYM_PATH

# This may take a few minutes
if not os.path.exists("PointCloudAlchemist"):
    !git clone https://github.com/Joohwan-Seo/CS282A_final_project.git PointCloudAlchemist
else:
    print("Already downloaded.")

%cd PointCloudAlchemist

/content/gdrive/My Drive/PointNet
Already downloaded.
/content/gdrive/My Drive/PointNet/PointCloudAlchemist


In [7]:
!pwd

/content/gdrive/My Drive/PointNet/PointCloudAlchemist


In [8]:
from utils import PointCloudData, Normalize, RotateXYZ, AddGaussianNoise, read_off, view_mesh, view_scatter
from network_sol import PointNetClassification, PointNetSegmentation, GetModel, PointNetLoss, TNet, Transform_Classification, Transform_Segmentation
from train_sol import TrainModel

## visualize image

The orignal dataset has objects given in a mesh form. Here is a visualisation of a chair in the original dataset.

In [9]:
verts, faces = read_off("data_for_viz/chair_0003.off")

In [10]:
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T

In [11]:
view_mesh(x,y,z, i,j,k )

Now we turn the mesh objects into a point cloud using three different sampling methods. We can sample points randomly from the mesh faces, here is what that point cloud looks like.

In [12]:
default = np.load("data_for_viz/chair_default.pkl", allow_pickle=True)
view_scatter(*default)

Here is the result of the original sampling method as proposed by the paper, where points are sampled from the faces of the objects.

In [13]:
original = np.load("data_for_viz/chair_original.pkl", allow_pickle=True)
view_scatter(*original)

Here is the point cloud from sampling based on the farthest point, where we find the centroid of the mesh surface and sample the farthest point from it, we iterate this to get the following point cloud.

In [14]:
farthest = np.load("data_for_viz/chair_farthest.pkl", allow_pickle=True)
view_scatter(*farthest)

### Implement transforms function 
We first define the data transforms, which is called by `torch.DataLoader`.
The transforms normalize, rotate and add noise to the pointclouds.

In [15]:
pointnet_tf = transforms.Compose([Normalize(),
                                  RotateXYZ(),
                                  AddGaussianNoise(),
                                  transforms.ToTensor()
                                ])

In [16]:
torch.manual_seed(340420)

method = 'random'

all_data = {'train': PointCloudData(data_split = "train", method = method, acc_type = 'valid' , transform = pointnet_tf),
            'test' : PointCloudData(data_split = "test",  method = method, acc_type = 'valid' , transform = pointnet_tf)
            } 

all_dataloader = {'train': DataLoader(dataset = all_data['train'], batch_size = 32, shuffle = True),
                  'test' : DataLoader(dataset = all_data['test'] , batch_size = 64)
                 }

The details of the dataset can be found in the following block.

In [17]:
num_to_class = {i: cat for cat, i in all_data['train'].classes.items()}

for t in ['train', 'test']:
    print(f"# of {t} data = {len(all_data[t])}")

print(f"# of classes: {len(all_data['train'].classes)}")

any_idx = 3990
assert any_idx < len(all_data['train'])

print(f"Sampled pointcloud shape: {all_data['train'][any_idx]['pointcloud'].size()}")
print(f"Class of the {any_idx}-th train data = {num_to_class[all_data['train'][any_idx]['category']]}")

# of train data = 3991
# of test data = 908
# of classes: 10
Sampled pointcloud shape: torch.Size([1024, 3])
Class of the 3990-th train data = toilet


# Write down core functions
Finish `network.py` and `train.py` - there are detailed comments in the code.


## Implement T-net in the `network.py`

Please refer to Fig.4 in the pdf file of the homework for the detailed architecture.

In [18]:
def test_Tnet():
  temp = TNet(3)
  torch.manual_seed(282)

  for layer in [temp.conv1, temp.conv2, temp.conv3, temp.fc1, temp.fc2, temp.fc3]:
    new_weight = torch.randn_like(layer.weight)
    layer.weight.data = new_weight
    new_bias = torch.randn_like(layer.bias)
    layer.bias.data = new_bias

  vec_1 = torch.randn((2,3,2))
  vec_2 = torch.randn((2,3,3))

  test_1 = temp(vec_1)
  test_2 = temp(vec_2)

  return test_1, test_2

test_1, test_2 = test_Tnet()

ans_1 = torch.tensor([[[-15.9464, -14.2467,  15.9300],
                       [ 14.2521,  -5.0106,  -0.2861],
                       [ -3.1092, -19.4553,   1.2760]],
                      [[ 15.4980,  -5.6156,   8.9752],
                       [  3.5774,   2.5102,   2.3002],
                       [ -6.5145,  -2.5212, -12.5888]]])

ans_2 = torch.tensor([[[ 10.8568,  -7.5004,  20.2903],
                       [ 12.6229,  -4.8022, -10.7242],
                       [ 10.1025, -13.6855,  -7.8211]],
                      [[-11.3499, -12.2782,   4.6686],
                       [  5.1668,   2.1487,  12.7927],
                       [-19.7324,  -8.4181,  -3.5816]]])

assert torch.allclose(test_1, ans_1, rtol=1e-3)
assert torch.allclose(test_2, ans_2, rtol=1e-3)

print("You pass the test!!")

You pass the test!!


## Implement `Transform_Classification` and `PointNetClassification` in `network.py`

Please refer to Fig.5 in the pdf file of the homework for the detailed architecture.

In [19]:
def test_Transfrom_Classification():
  temp = Transform_Classification(False)
  torch.manual_seed(282)

  for layer in [temp.conv1, temp.conv2, temp.conv3]:
    new_weight = torch.randn_like(layer.weight)
    layer.weight.data = new_weight
    new_bias = torch.randn_like(layer.bias)
    layer.bias.data = new_bias

  vec_1 = torch.randn((2,3,2))
  test_1 = temp(vec_1)

  return test_1[0][:, :5].data

test_1 = test_Transfrom_Classification()

ans_1 = torch.tensor([[0.3051, 1.5555, 1.1269, 0.5964, 1.5081],
                      [1.1478, 0.0618, 0.7057, 1.2762, 0.0065]])

assert torch.allclose(test_1, ans_1, rtol=1e-2)

print("You pass the test!!")

You pass the test!!


## Implement `Transform_Segmentation` and `PointNetSegmentation` in `network.py`

Please refer to Fig.6 in the pdf file of the homework for the detailed architecture.


In [20]:
def test_Transform_Segmentation():
  temp = Transform_Segmentation(False)
  torch.manual_seed(282)

  for layer in [temp.conv1, temp.conv2, temp.conv3, temp.conv4, temp.conv5]:
    new_weight = torch.randn_like(layer.weight)
    layer.weight.data = new_weight
    new_bias = torch.randn_like(layer.bias)
    layer.bias.data = new_bias

  vec_1 = torch.randn((1, 3, 4))
  test_1, _, _ = temp(vec_1)

  return test_1[0].squeeze()[:5, :]

test_1 = test_Transform_Segmentation()

ans_1 = torch.tensor([[1.5396, 0.0000, 0.2349, 0.0000],
                      [0.0000, 0.0000, 0.0000, 1.7139],
                      [0.8661, 0.0000, 0.0881, 0.7033],
                      [0.0000, 1.5564, 0.0000, 0.0000],
                      [0.0000, 0.0000, 0.0000, 1.7158]])

assert torch.allclose(test_1, ans_1, rtol=1e-2)

print("You pass the test!!")

You pass the test!!


# Model Training & Testing
Now, we will test the training results under several conditions. 
In particular, the questions we want to answer is as follows:
- What is the role of the Tnet?
- How does the data sampling method affect the performance?

To answer this question, we will train 4 classification models: with/without Tnet, random/farthest sampling method.

For the segmentation tasks, we fix the sampling method as random, but we are only checking with with/without Tnet.



In [21]:
# If you rerun this cell, you will lost all your work
model_dict, output_dict = {}, {}
for tnet in ['yes_t', 'no_t']:
    model_dict[tnet], output_dict[tnet] = {}, {}
    for task in ['Classification', 'Segmentation']:
        model_dict[tnet][task], output_dict[tnet][task] = {}, {}
        for method in ['random', 'farthest']:
            model_dict[tnet][task][method], output_dict[tnet][task][method] = None, None

## Classification Model training
### Case 1: With Tnet / Random



In [ ]:
tnet = 'yes_t' 
task = 'Classification'
method = 'random'

# set model and check device
model_dict[tnet][task][method], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task][method].parameters()).device)

# train
output_dict[tnet][task][method] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task][method],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

### Case 2: Without Tnet / Random

In [ ]:
tnet = 'no_t' 
task = 'Classification'
method = 'random'

# set model and check device
model_dict[tnet][task][method], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task][method].parameters()).device)

# train
output_dict[tnet][task][method] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task][method],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

### Re-defining Dataloaders

In [24]:
method = 'farthest'

all_data = {'train': PointCloudData(data_split = "train", method = method, acc_type = 'valid' , transform = pointnet_tf),
            'test' : PointCloudData(data_split = "test",  method = method, acc_type = 'valid' , transform = pointnet_tf)
            } 

all_dataloader = {'train': DataLoader(dataset = all_data['train'], batch_size = 32, shuffle = True),
                  'test' : DataLoader(dataset = all_data['test'] , batch_size = 64)
                 }

## Case 3: With Tnet / Farthest

In [ ]:
tnet = 'yes_t' 
task = 'Classification'
method = 'farthest'

# set model and check device
model_dict[tnet][task][method], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task][method].parameters()).device)

# train
output_dict[tnet][task][method] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task][method],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

## Case 4: Without Tnet / Farthest

In [ ]:
tnet = 'no_t' 
task = 'Classification'
method = 'farthest'

# set model and check device
model_dict[tnet][task][method], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task][method].parameters()).device)

# train
output_dict[tnet][task][method] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task][method],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

## Part segmentation models training


Re-defining dataloader for the Segmentation tasks

In [27]:
all_data = {'train': PointCloudData(data_split = "train", method = 'random', task = 'Segmentation', acc_type = 'valid' , transform = pointnet_tf),
            'test' : PointCloudData(data_split = "test",  method = 'random', task = 'Segmentation', acc_type = 'valid' , transform = pointnet_tf)
            } 
all_dataloader = {'train': DataLoader(dataset = all_data['train'], batch_size = 32, shuffle = True),
                  'test' : DataLoader(dataset = all_data['test'] , batch_size = 64)
                 }

## Case 1: With Tnet / Random

In [ ]:
tnet = 'yes_t' 
task = 'Segmentation'
method = 'random'

# set model and check device
model_dict[tnet][task][method], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task][method].parameters()).device)

# train
output_dict[tnet][task][method] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task][method],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

## Case 2: With Tnet / Random

In [ ]:
tnet = 'no_t' 
task = 'Segmentation'
method = 'random'

# set model and check device
model_dict[tnet][task][method], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task][method].parameters()).device)

# train
output_dict[tnet][task][method] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task][method],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

# Plotting for the training loss and validation accuracy
## Classification

In [ ]:
arg_tnet = ['yes_t', 'no_t']
arg_method = ['farthest', 'random']

# For the classification task
task = 'Classification'
k = 1
plt.figure(1, figsize = (12,8))
legend = []
for tnet in arg_tnet:
  for method in arg_method:
    plt.subplot(2,1,1)
    data = output_dict[tnet][task][method]
    train_loss = []
    acc = []
    N = len(data)
    for i in range(N):
      train_loss.append(np.mean(data[i]['train_loss']))
      acc.append(data[i]['accuracy'])

    plt.subplot(2,1,1)
    plt.plot(range(N), train_loss)
    plt.subplot(2,1,2)
    plt.plot(range(N), acc)
    legend.append(tnet + ',' + method)

plt.subplot(2,1,1)
plt.legend(legend);
plt.ylabel('Training Loss');
plt.subplot(2,1,2)
plt.legend(legend);
plt.ylabel('Validation Accuracy');
plt.xlabel('num epochs');

    # break

## Segmentation

In [ ]:
arg_tnet = ['yes_t', 'no_t']
arg_method = ['random']

# For the classification task
task = 'Segmentation'
k = 1
plt.figure(1, figsize = (12,8))
legend = []
for tnet in arg_tnet:
  for method in arg_method:
    plt.subplot(2,1,1)
    data = output_dict[tnet][task][method]
    train_loss = []
    acc = []
    N = len(data)
    for i in range(N):
      train_loss.append(np.mean(data[i]['train_loss']))
      acc.append(data[i]['accuracy'])

    plt.subplot(2,1,1)
    plt.plot(range(N), train_loss)
    plt.subplot(2,1,2)
    plt.plot(range(N), acc)
    legend.append(tnet + ',' + method)

plt.subplot(2,1,1)
plt.legend(legend);
plt.ylabel('Training Loss');
plt.subplot(2,1,2)
plt.legend(legend);
plt.ylabel('Validation Accuracy');
plt.xlabel('num epochs');

**Question 5. Attach the training loss and the validation accuracy plot.**

# Test accuracy


In [32]:
tnet = 'yes_t' 
task = 'Segmentation'
method = 'random'

mymodel = model_dict[tnet][task][method]
mymodel.eval();

In [ ]:
test_data = PointCloudData(data_split = "test",  method = method, task = task, acc_type = 'test' , transform = None)
test_loader = DataLoader(dataset = test_data , batch_size = 64)

with torch.no_grad():
  correct, total = 0, 0
  k = 1
  for data in test_loader:
      # get pointcloud and category
      pointcloud = data['pointcloud'].to(DEVICE).float()
      # print(pointcloud.shape)
      category   = data['category'].to(DEVICE)
      # run the model
      output, _, _ = mymodel(pointcloud.transpose(1,2))
      _, predict_category = torch.max(output.data, 1)
      # record the correct predicitons
      if task == "Classification":
        total += predict_category.size(0)
      elif task == "Segmentation":
        total += predict_category.size(0) * predict_category.size(1)  
      correct += (predict_category == category).sum().item()

accuracy = int((correct / total) * 100)
print(f"Test accuracy: {accuracy}")